In [1]:
from collections import defaultdict
import json
import math

# Load Equipments

In [2]:
equipments = json.load(open("data/equipment.json","r", encoding="utf8"))
PART_NAMES = ['tou', 'wan', 'xiong', 'yao', 'jiao']
ALL_EQUIPMENTS = {}
for part in PART_NAMES:
    items = []
    for item in equipments["data"]:
        if item["buwei"] == part:
            skill_dict = {}
            for skill in item["skillArray"]:
                skill_dict[skill["skillId"]] = skill.get("skillNum", 0)
            name = "%s (%s/%s)" % (item["name_cn"], item["name"], item["name_en"])
            items.append([name, skill_dict, item["hole"], item["type"], item["initDefense"], item["maxDefense"]])
    ALL_EQUIPMENTS[part] = items
    print("loaded %s parts: %d" % (part, len(items)))

loaded tou parts: 1211
loaded wan parts: 1168
loaded xiong parts: 1191
loaded yao parts: 1118
loaded jiao parts: 1178


# Load Skill and Activation

In [3]:
skills = json.load(open("data/skill_activation.json","r", encoding="utf8"))
SKILL_NAME2ID = {}
SKILL_ID2NAME = {}
ACTIVATION_TO_SKILL = {}
for item in skills["data"]:
    SKILL_NAME2ID[item["skillPointName_cn"]] = item["id"]
    SKILL_ID2NAME[item["id"]] = item["skillPointName_cn"]
    for activation in item["skillFadongArray"]:
        if int(activation["num"]) > 0:
            ACTIVATION_TO_SKILL[activation["skillFadong_cn"]] = (item["id"], int(activation["num"]))

In [4]:
print("技能名称:", list(SKILL_NAME2ID.keys()))

技能名称: ['攻击', '穷地', '英雄之盾', '灭气攻击', '冰属性攻击', '逆境', '拔刀灭气', '水属性攻击', '蓄力缩短', '属性攻击', '特殊会心', '炮术', '雷属性攻击', '龙属性攻击', '属性会心', '火属性攻击', '连击', '痛击', '防御', '守护', '纳刀研磨', '钝器', '认真', '爆弹强化', '肉食', '刃鳞', '忍耐', '会心強化', '研磨术', '根性', '耐力', '特殊攻击', '锋利度', '收刀', 'ＳＰ延长', '持续行走', '大胃王', '进食', '底力', '剑术', '气力回复', '龙气', '达人', '拔刀会心', '重击', '斗魂', '体力', '饥饿感', '泡沫', '无心', '防御强化', '里会心', '骑乘', '体术', '茸食', '匠', '无伤', '逆上', '回避距离', '防御性能', 'KO', '跳跃', '回避性能', '机会', '狂击耐性', '装填数', '蜂蜜', '毒', '麻痹瓶追加', '水耐性', '回复量', '雷耐性', '火耐性', '对防御DOWN', '风压', '接击瓶追加', '炎热适应', '节食', '雪人', '榴弹追加', '护石强化', '耐暑', '灭气瓶追加', '裂伤', '剥取', '搬运', '耐震', '冰耐性', '细菌学', '贯通弹追加', '偷盗无效', '通常弹强化', '散弹追加', '强击瓶追加', '装填速度', '弹药节约', '反复无常', '扩散弹追加', '射法', '调合数', '回复速度', '广域', '属强瓶追加', '毒瓶追加', '龙耐性', '速射', '护石王', '寒冷适应', '效果持续', '变则射击', '睡眠瓶追加', '爆破瓶追加', '运气', '野草知识', '听觉保护', '反动', '属性耐性', '耐寒', '贯通弹强化', '重击弹强化', '麻痹', '睡眠', '晕眩', '通常弹追加', '散弹强化', '精密射击', '捕获', '采集', '刚击', '贝尔纳', '天眼', '银岭', '真・岩穿', '真・铠裂', '警惕', '观察眼', '猎人', '怒', '增幅', '对炎龙',

In [5]:
print("发动技能名称:", list(ACTIVATION_TO_SKILL.keys()))

发动技能名称: ['攻击力UP【大】', '攻击力UP【中】', '攻击力UP【小】', '绝境求生', '英雄守护', '耐力夺取', '冰属性攻击强化+2', '冰属性攻击强化+1', '不屈', '拔刀术【力】', '水属性攻击强化+2', '水属性攻击强化+1', '集中', '属性攻击强化', '会心击【特殊】', '炮术王', '炮术师', '雷属性攻击强化+2', '雷属性攻击强化+1', '龙属性攻击强化+2', '龙属性攻击强化+1', '会心击【属性】', '火属性攻击强化+2', '火属性攻击强化+1', '连击的心得', '弱点特效', '防御力UP【大】', '防御力UP【中】', '防御力UP【小】', '精灵加护', '挑战者的纳刀', '钝器使用', '力量解放+2', '力量解放+1', '爆弹人', '最爱吃肉', '刃鳞研磨', '虎视眈眈', '超会心', '刚刃研磨', '根性', '飞人', '状态异常攻击+2', '状态异常攻击+1', '利刃', '纳刀术', 'ＳＰ时间延长', '持续奔跑', '饥不择食', '饱食', '速食者+2', '速食者+1', '火事场力+2', '火事场力+1', '心眼', '耐力急速回复', '龙气活性', '洞悉+3', '洞悉+2', '洞悉+1', '拔刀术【技】', '破坏王', '挑战者+2', '挑战者+1', '体力+50', '体力+20', '饥饿感无效', '饥饿感减半', '泡沫之舞', '明镜止水', '防御强化', '痛恨会心', '骑乘名人', '体术+2', '体术+1', '最爱蘑菇', '锋利度等级+2', '锋利度等级+1', '全蓄力', '仇恨', '回避距离UP', '防御性能+2', '防御性能+1', 'KO术', '飞燕', '回避性能+2', '回避性能+1', '王牌', '无我的境地', '装填数UP', '蜂蜜猎人', '毒耐性', '麻痹瓶追加', '水耐性【大】', '水耐性【小】', '体力回复量UP', '雷耐性【大】', '雷耐性【小】', '火耐性【大】', '火耐性【小】', '铁面皮', '风压【大】无效', '风压【小】无效', '接击瓶追加', '南风的狩人', '满足感', '雪人无效', '穿甲榴弹全L

# Load Gems

In [6]:
gems = json.load(open("data/gems.json","r", encoding="utf8"))
ALL_GEMS = []
for gem in gems["data"]:
    skills = {}
    if gem["effect1"] in SKILL_NAME2ID:
        skills[SKILL_NAME2ID[gem["effect1"]]] = gem["effect1_num"]
    if gem["effect2"] in SKILL_NAME2ID:
        skills[SKILL_NAME2ID[gem["effect2"]]] = gem["effect2_num"]
    ALL_GEMS.append([gem["name_cn"], skills, gem["hole"]])

# Input Requirement

In [7]:
# 需要的技能 从上面发动技能名称中粘贴
required_activated_skills = [
    '洞悉+2',
    '弱点特效',
    '贯通弹・贯通箭UP',
    '超会心',
    '弹导强化'
]
# 护石提供的技能点 从上面技能名称中粘贴
stone_skills = {
    "痛击": 6
}
# 护石孔数
stone_hole = 3
# 武器孔数
weapon_hole = 1
# 是否为远程: 远程为1 近战为0 
is_gunner = 1

# Processing

In [8]:
required_point_dict = defaultdict(int)
for activated_skill in required_activated_skills:
    required_point_dict[ACTIVATION_TO_SKILL[activated_skill][0]] += ACTIVATION_TO_SKILL[activated_skill][1]

id2dim = {}
required_skills = []
for key in required_point_dict.keys():
    id2dim[key] = len(required_skills)
    required_skills.append(key)
    
required_points = [0] * len(required_skills)
stone_points = [0] * len(required_skills)
for key in required_point_dict.keys():
    required_points[id2dim[key]] = required_point_dict[key]
for key in stone_skills:
    stone_points[id2dim[SKILL_NAME2ID[key]]] = stone_skills[key]

for i in range(len(required_skills)):
    print("%s:需要%d点 护石提供%d点" % (SKILL_ID2NAME[required_skills[i]], required_points[i], stone_points[i]))
print("武器%d孔 护石%d孔" % (weapon_hole, stone_hole))

达人:需要15点 护石提供0点
痛击:需要10点 护石提供6点
贯通弹强化:需要10点 护石提供0点
会心強化:需要10点 护石提供0点
射法:需要10点 护石提供0点
武器1孔 护石3孔


In [9]:
def isDominate(la, lb):
    for a, b in zip(la, lb):
        if a < b:
            return False
    return True

def search_candidate(input_items, required_skills, require_type):
    selected_items = []
    for item in input_items:
        if require_type != -1 and item[-3] !=  require_type:
            continue
        current_item = []
        total = 0
        for s in required_skills:
            skill_num = item[1].get(s, 0)
            total += skill_num
            current_item.append(skill_num)
        current_item.append(item[2])

        if total == 0:
            continue

        is_dominated = False
        for sitem in selected_items:
            if isDominate(sitem[1], current_item):
                is_dominated = True
                break

        if is_dominated:
            continue

        selected_items = [sitem for sitem in selected_items if not isDominate(current_item, sitem[1])]
        selected_items.append([item[0], current_item, item[-2], item[-1]])
    return selected_items

In [10]:
def prefilter_gem(required_skills):
    prefiltered_gems = []
    for gem in ALL_GEMS:
            is_useful = False
            for skill in required_skills:
                if gem[1].get(skill, 0) > 0:
                    is_useful = True
                    break
            if is_useful:
                prefiltered_gems.append(gem)
    return prefiltered_gems

def prepare_gem_search(equipments, required_skills, required_points, stone_points, stone_hole, weapon_hole, prefilter_gems):
    hole_counts = [0] * 4 # holes for 1, 2, 3
    hole_counts[stone_hole] += 1
    hole_counts[weapon_hole] += 1
    
    left_over_points = required_points[:]
    num_skills = len(left_over_points)
    
    # count stone skill
    for i in range(num_skills):
        left_over_points[i] -= stone_points[i]
        
    for equip in equipments:
        hole_counts[equip[1][-1]] += 1
        for i in range(num_skills):
            left_over_points[i] -= equip[1][i]
    
    # new requirement
    new_required_skills = []
    new_required_points = []
    for i in range(num_skills):
        if left_over_points[i] > 0:
            new_required_skills.append(required_skills[i])
            new_required_points.append(left_over_points[i])
    
    # filter useful gems
    useful_gems = []
    max_single_gain = [0.0] * len(new_required_skills)
    for gem in prefilter_gems:
        is_useful = False
        gen_skill = [0] * len(new_required_skills)
        for i, skill in enumerate(new_required_skills):
            p = gem[1].get(skill, 0)
            if p > 0:
                is_useful = True
                gen_skill[i] = p
                max_single_gain[i] = max(max_single_gain[i], 1.0 * p / gem[2])
        if is_useful:
            useful_gems.append([gem[0], gen_skill, gem[2]])
    
    # greedy prune
    total_holes = hole_counts[1] + 2* hole_counts[2] + 3 * hole_counts[3]
    for require, gain in zip(new_required_points, max_single_gain):
        require_num = int(math.ceil(require / gain))
        total_holes -= require_num
        if total_holes < 0:
            return False, None, None, None, None
    
    return True, [max(v, 0) for v in left_over_points], hole_counts, new_required_points, useful_gems

In [11]:
def search_gem(required_points, hole_counts, useful_gems, result, cache):
    is_done = True
    for v in required_points:
        if v > 0:
            is_done = False
            break
    if is_done:
        return True
    
    state = tuple(required_points + hole_counts)
    if state in cache:
        return cache[state]
    
    for gem in useful_gems:
        is_useful = False
        for i, v in enumerate(gem[1]):
            if required_points[i] > 0 and v > 0:
                is_useful = True
                break
        if is_useful:
            for h in range(1, 4):
                if gem[2] <= h and hole_counts[h] > 0:
                    hole_counts[h] -= 1
                    hole_counts[h - gem[2]] += 1
                    
                    for i, v in enumerate(gem[1]):
                        required_points[i] -= v

                    result.append(gem[0])
                    cache[state] = search_gem(required_points, hole_counts, useful_gems, result, cache)
                    if cache[state]:
                        return True
                    result.pop()
                    for i, v in enumerate(gem[1]):
                        required_points[i] += v
                    
                    hole_counts[h] += 1
                    hole_counts[h - gem[2]] -= 1
    return False

In [12]:
tou_equips = sorted(search_candidate(ALL_EQUIPMENTS["tou"], required_skills, -1), key=lambda x: -x[-1])
wan_equips = sorted(search_candidate(ALL_EQUIPMENTS["wan"], required_skills, is_gunner), key=lambda x: -x[-1])
xiong_equips = sorted(search_candidate(ALL_EQUIPMENTS["xiong"], required_skills, is_gunner), key=lambda x: -x[-1])
yao_equips = sorted(search_candidate(ALL_EQUIPMENTS["yao"], required_skills, is_gunner), key=lambda x: -x[-1])
jiao_equips = sorted(search_candidate(ALL_EQUIPMENTS["jiao"], required_skills, is_gunner), key=lambda x: -x[-1])

possible_results = []
prefilered_gems = prefilter_gem(required_skills)
for tou in tou_equips:
    for wan in wan_equips:
        for xiong in xiong_equips:
            for yao in yao_equips:
                for jiao in jiao_equips:
                    equipments = [tou, wan, xiong, yao, jiao]
                    is_possible, left_over_points, hole_counts, new_required_points, useful_gems = prepare_gem_search(equipments, required_skills, required_points, stone_points, stone_hole, weapon_hole,prefilered_gems)
                    if is_possible:
                        result_gems = []
                        cahce = {}
                        useful_gems = sorted(useful_gems, key=lambda x: -x[2])
                        if search_gem(new_required_points, hole_counts, useful_gems, result_gems, cahce):
                            print("=============================总防御力: %d -> %d=============================" % (
                                tou[-2]+wan[-2]+xiong[-2]+yao[-2]+jiao[-2],
                                tou[-1]+wan[-1]+xiong[-1]+yao[-1]+jiao[-1]
                            ))
                            print("头: %s 防御 %d -> %d" % (tou[0], tou[-2], tou[-1]))
                            print("腕: %s 防御 %d -> %d" % (wan[0], wan[-2], wan[-1]))
                            print("胸: %s 防御 %d -> %d" % (xiong[0], xiong[-2], xiong[-1]))
                            print("腰: %s 防御 %d -> %d" % (yao[0], yao[-2], yao[-1]))
                            print("脚: %s 防御 %d -> %d" % (jiao[0], jiao[-2], jiao[-1]))
                            print("珠子: %s" % ", ".join(result_gems))
                            print("")

=============================总防御力: 312 -> 439=============================
头: 隼刃羽饰 (隼刃の羽飾り/Hayabusa Feather) 防御 20 -> 51
腕: 海龙护手X (ラギアXガード/Lagiacrus Guards X) 防御 72 -> 96
胸: 曙丸战流【胴甲】 (曙丸戦流【胴当て】/Battlefield Genesis Torso) 防御 69 -> 93
腰: 银火龙裙甲XR (S・ソルXRコート/Silver Solcoat XR) 防御 75 -> 99
脚: 霸龙轻靴XR (アカムトXRチキル/Akantor Boots XR) 防御 76 -> 100
珠子: 射法珠【2】, 射法珠【2】, 射法珠【2】, 射法珠【1】, 会心珠【1】, 会心珠【1】

=============================总防御力: 309 -> 436=============================
头: 隼刃羽饰 (隼刃の羽飾り/Hayabusa Feather) 防御 20 -> 51
腕: 幸福护手X (チアフルXガード/Cheerful Guards X) 防御 69 -> 93
胸: 曙丸战流【胴甲】 (曙丸戦流【胴当て】/Battlefield Genesis Torso) 防御 69 -> 93
腰: 银火龙裙甲XR (S・ソルXRコート/Silver Solcoat XR) 防御 75 -> 99
脚: 霸龙轻靴XR (アカムトXRチキル/Akantor Boots XR) 防御 76 -> 100
珠子: 贯通珠【３】, 射法珠【2】, 射法珠【2】, 会心珠【1】, 会心珠【1】

